In [2]:
from haystack import Pipeline, component
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack.utils import Secret
from haystack.components.builders import ChatPromptBuilder, PromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.tools.tool import Tool
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.retrievers import ChatMessageRetriever
from haystack_experimental.components.writers import ChatMessageWriter
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.generators import OpenAIGenerator
from pymongo import MongoClient
from typing import List,Annotated
from haystack.components.routers import ConditionalRouter
from haystack.dataclasses import Document
from getpass import getpass
import re
import json
import os

d:\Enviroments\anaconda3\envs\day45-haystack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### LOGGER

In [3]:
import logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.DEBUG)

tracing.tracer.is_content_tracing_enabled = True # to enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"}))

### ENV VARIABLES

In [4]:
mongo_connection_string = Secret.from_token(os.getenv("MONGO_CONNECTION_STRING"))
openai_api_key = Secret.from_token(os.getenv("OPENAI_API_KEY"))

In [5]:
chat_message_store = InMemoryChatMessageStore()
document_store_product = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index",
    full_text_search_index="search_index",
    mongo_connection_string=mongo_connection_string
)
document_store_common = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="common_informataion",
    vector_search_index="vector_index_ccommon",
    full_text_search_index=None,
    mongo_connection_string=mongo_connection_string
)

### Routing Variable

In [6]:
routes = [
    {
        "condition": "{{ replies[0] == 'products'}}", 
        "output": ["{{ query_embedding }}","{{ query }}"],             
        "output_name": ["products_route_embedding" , "products_route_query"],        
        "output_type": [list[float], str]             
    },
    {
        "condition": "{{ replies[0] == 'common'}}", 
        "output": ["{{ query_embedding }}","{{ query }}"],
        "output_name": ["common_route_embedding" , "common_route_query"],
        "output_type": [list[float], str]
    }
]
router = ConditionalRouter(routes)

## Membuat Paraphraser Tool

In [ ]:
class ParaphraserPipeline:
    def __init__(self,chat_message_store):
        self.memory_retriever = ChatMessageRetriever(chat_message_store)
        self.pipeline = Pipeline()
        self.pipeline.add_component("prompt_builder",ChatPromptBuilder(variables=["query","memories"],required_variables=["query", "memories"],))
        self.pipeline.add_component("generator", OpenAIChatGenerator(model="gpt-4.1-2025-04-14", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))
        self.pipeline.add_component("memory_retriever", self.memory_retriever)

        self.pipeline.connect("prompt_builder.prompt", "generator.messages")
        self.pipeline.connect("memory_retriever", "prompt_builder.memories")
    
    def run(self, query):
        messages = [
            ChatMessage.from_system(
                "You are a helpful assistant that paraphrases user queries based on previous conversations."
            ),
            ChatMessage.from_user(
                """
                Please paraphrase the following query based on the conversation history provided below. If the conversation history is empty, please return the query as is.
                history:
                {% for memory in memories %}
                    {{memory.text}}
                {% endfor %}
                query: {{query}}
                answer:
                """
            )
        ]

        res = self.pipeline.run(
            data = {
                "prompt_builder":{
                    "query": query,
                    "template": messages
                },
            
            },
            include_outputs_from=["generator"]
        )
        print("Pipeline Input", query)
        return res["generator"]["replies"][0].text

## Membuat History Tool

In [8]:
class ChatHistoryPipeline:
    def __init__(self, chat_message_store):
        self.chat_message_store = chat_message_store
        self.pipeline = Pipeline()
        self.pipeline.add_component("memory_retriever", ChatMessageRetriever(chat_message_store))
        self.pipeline.add_component("prompt_builder", PromptBuilder(variables=["memories"], required_variables=["memories"], template="""
        Previous Conversations history:
        {% for memory in memories %}
            {{memory.text}}
        {% endfor %}
        """)
        )
        self.pipeline.connect("memory_retriever", "prompt_builder.memories")

    def run(self):
        res = self.pipeline.run(
            data = {},
            include_outputs_from=["prompt_builder"]
        )

        return res["prompt_builder"]["prompt"]

## Membuat routing tools

In [ ]:

class RoutingPipeline: 
    def __init__(self, router):
        self.router = router
        self.pipeline = Pipeline()

        router_prompt_template = [
            ChatMessage.from_system( """
                You are a routing classifier.
                Return EXACTLY ONE of the following strings:
                products
                common

                STRICT RULES:
                - DO NOT add quotes
                - DO NOT add punctuation
                - DO NOT add explanation
                - DO NOT add extra words
                - ONLY return exactly: products OR common

                Examples:
                Query: What is the price of this bag
                Answer: products

                Query: How do I return my order
                Answer: common

                Query: Does this bag come in red
                Answer: products

                Query: I want a refund for my last order
                Answer: common"
                """
            ),
            ChatMessage.from_user("{{ query }}")
       ]

        self.pipeline.add_component("embedder", SentenceTransformersTextEmbedder())
        self.pipeline.add_component("decision", OpenAIChatGenerator(model="gpt-4.1", api_key=openai_api_key))        
        self.pipeline.add_component("prompt_builder_router", ChatPromptBuilder(template=router_prompt_template))
        self.pipeline.add_component("router", router)
        
        self.pipeline.connect("embedder.embedding", "router.query_embedding")
        self.pipeline.connect("prompt_builder_router.prompt", "decision")
        self.pipeline.connect("decision.replies", "router.replies")

    def run(self, query):
        
        res = self.pipeline.run(
            data={
                "prompt_builder_router": {"query" : query},
                "embedder": {"text" : query},
                "router": {"query": query}
            }, 
            include_outputs_from=["router", "decision"]
        )
        try:
            return res["decision"]["replies"][0]
        except (KeyError, IndexError):
            return "routing_failed"

## Membuat Route Common

In [ ]:
class CommonRoute:
    def __init__(self, document_store_common):
        self.document_store_product = document_store_common
        template_common_message = [
            ChatMessage.from_system(
            """
            You are smart personal assistant system to help customer find common information
            Answer the user's question using only the provided context.
            Maintain the same language as the question.   
            Context:
            {{ context | map(attribute='content') | join(" ") | replace("\n", " ")}}   
            Instructions:
            1. Only use information from the context to answer.
            2. If the context does not contain the required Context, respond with:
            "I'm sorry, I can't answer that right now."
            3. Keep the answer concise and clear.          
            """
        ),
            ChatMessage.from_user(
                "{{ query }}"
            )
        ]
        self.pipeline = Pipeline()
        self.pipeline.add_component("retriever_common", MongoDBAtlasEmbeddingRetriever( document_store=document_store_common, top_k=6))
        self.pipeline.add_component("prompt_builder_common", ChatPromptBuilder(template=template_common_message))
        self.pipeline.add_component("generator_common", OpenAIChatGenerator(api_key=openai_api_key, model="gpt-4.1"))

        self.pipeline.connect("retriever_common.documents", "prompt_builder_common.documents")
        self.pipeline.connect("prompt_builder_common.prompt", "generator_common.prompt")
    def run(self, query_embedding, query):
        res = self.pipeline.run(
            data={
                "prompt_builder_common":{"query" : query},
                "retriever_common":{"query_embedding" : query_embedding}
            }, include_outputs_from=["generator_common"]
        )
        return res["generator"]["replies"][0]